In [11]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import pyarrow.parquet as pq
import gcsfs

In [12]:
files = ['green_tripdata_2020-10.csv', 'green_tripdata_2020-11.csv', 'green_tripdata_2020-12.csv']

df_list = [pd.read_csv(file, low_memory=False) for file in files]
df = pd.concat(df_list, ignore_index=True)

# Question 1

In [13]:
print("Shape of the DataFrame after loading and concatenation:", df.shape)

Shape of the DataFrame after loading and concatenation: (266855, 20)


# Question 2

In [14]:
df_filtered = df[(df['passenger_count'] > 0) & (df['trip_distance'] > 0)].copy()

In [15]:
print(f"Number of rows after filtering: {df_filtered.shape[0]}")

Number of rows after filtering: 139370


# Question 3

In [16]:
data['lpep_pickup_date'] = data['lpep_pickup_datetime'].dt.date

NameError: name 'data' is not defined

# Question 4

In [17]:
unique_vendor_ids = df['VendorID'].unique()
print(unique_vendor_ids)

[ 2.  1. nan]


# Question 5

In [19]:
import re

def is_not_snake_case(col_name):
    return bool(re.match(r'.*[A-Z ]', col_name))

columns_not_snake_case = [col for col in df.columns if is_not_snake_case(col)]
num_columns_not_snake_case = len(columns_not_snake_case)

print(f"Number of columns to be renamed to snake case: {num_columns_not_snake_case}")

Number of columns to be renamed to snake case: 4


# Question 6

In [ ]:
gsutil ls gs://de_hw1/path/to/store/parquet_files/ | grep '/' | wc -l

# Answer:  95

In [3]:
df_filtered = df[(df['passenger_count'] > 0) & (df['trip_distance'] > 0)].copy()

df_filtered['lpep_pickup_date'] = pd.to_datetime(df_filtered['lpep_pickup_datetime']).dt.date

df_transformed = df_filtered.rename(columns=lambda x: x.lower().replace("id", "_id"))

assert df_transformed['vendor_id'].isin([1, 2]).all(), "vendor_id contains unexpected values"
assert (df_transformed['passenger_count'] > 0).all(), "There are rows with passenger_count <= 0"
assert (df_transformed['trip_distance'] > 0).all(), "There are rows with trip_distance <= 0"

In [4]:
username = 'postgres'
password = 'rabd9xYUU5'
hostname = 'localhost'
port = '5432'
database_name = 'green_taxi_data'

engine = create_engine(f'postgresql://{username}:{password}@{hostname}:{port}/{database_name}')

df_transformed.to_sql('green_taxi', con=engine, schema='mage', if_exists='replace', index=False)

370

In [5]:
engine = create_engine(f'postgresql://{username}:{password}@{hostname}:{port}/{database_name}')

query = "SELECT * FROM mage.green_taxi LIMIT 5;"
df = pd.read_sql_query(query, con=engine)

print(df.head())

   vendor_id lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0        2.0  2020-10-01 00:31:19   2020-10-01 00:34:55                  N   
1        2.0  2020-10-01 00:42:12   2020-10-01 00:43:51                  N   
2        2.0  2020-10-01 00:53:09   2020-10-01 00:55:39                  N   
3        1.0  2020-10-01 00:12:29   2020-10-01 00:20:08                  N   
4        1.0  2020-10-01 00:32:38   2020-10-01 00:43:02                  N   

   ratecode_id  pulocation_id  dolocation_id  passenger_count  trip_distance  \
0          1.0              7              7              1.0           0.79   
1          1.0            179              7              1.0           0.50   
2          1.0            179            223              1.0           0.60   
3          1.0            134            216              2.0           4.40   
4          1.0             82              7              1.0           2.90   

   fare_amount  ...  mta_tax  tip_amount  tolls_am

In [7]:
import pyarrow as pa
import pyarrow.parquet as pq
import gcsfs

bucket_name = 'de_hw1'
base_path = 'path/to/store/parquet_files/'

fs = gcsfs.GCSFileSystem()

def save_df_to_parquet_partitioned(df, column, bucket_name, base_path):
    for date, df_partition in df.groupby(column):
        file_path = f'{base_path}{date}.parquet'
        table = pa.Table.from_pandas(df_partition)
        with fs.open(f'gs://{bucket_name}/{file_path}', 'wb') as f:
            pq.write_table(table, f)

save_df_to_parquet_partitioned(df_transformed, 'lpep_pickup_date', bucket_name, base_path)

In [9]:
df.shape

(5, 21)